# Pre-processing the data - batch
This notebook follows on from `transform_and_preprocess_spice.ipynb`, so look at that first for an explanation of processing. This notebook has the same processing in fewer commands and in a loop across forecast ref times. Along the way intermediate data is also output, including:
* MOGREPS-G gridded data for UK, all variables in one file.
* MOGREPS-G data for UK in tabular form
* Radar data regridded onto MOGREPS-G grid and summed to 3hr accumualtions, one file
* Radar data on MOGREPS-G grid and summed to 3hr accumualtions in tabular form
* Final output: Merged MOGREPS-G and radar data in tabular

Changing the initial parameter (such as time and location) should allow duplicated o f this notebook to work for other timeperiods.

In [1]:
import pathlib
import datetime
import functools
import os

In [2]:
import numpy

In [3]:
import pandas

In [4]:
import xarray
import iris
import iris.quickplot
import iris.coord_categorisation

In [5]:
import matplotlib.pyplot

# Set parameters for notebook
Set the paths and lists of things to process

In [6]:
project_name = 'precip_rediagnosis'
mogreps_g_name = 'mogreps-g'
radar_name = 'radar'
ilab_project_dir = pathlib.Path('/project/informatics_lab/')
output_dir =  pathlib.Path('/scratch')/ os.environ['USER'] / project_name / 'storm_dennis_6hr_lt'

In [7]:
root_data_dir = pathlib.Path('/scratch/shaddad/precip_rediagnosis/storm_dennis_6hr_lt')

In [8]:
mogreps_g_data_dir = root_data_dir / mogreps_g_name
radar_data_dir = root_data_dir / radar_name

In [9]:
date_fname_template = '{start.year:04d}{start.month:02d}{start.day:02d}T{start.hour:02d}{start.minute:02d}Z_{end.year:04d}{end.month:02d}{end.day:02d}T{end.hour:02d}{end.minute:02d}Z'
fname_extension_grid = '.nc'
fname_extension_tabular = '.csv'
leadtime_template = '{lt:03d}H'
mogreps_g_tab_fname_template = 'prd_mogreps_g_' + leadtime_template + '_' + date_fname_template + fname_extension_tabular
mogreps_g_grid_fname_template = 'prd_mogreps_g_' + leadtime_template + '_' + date_fname_template + fname_extension_grid
radar_tab_fname_template = 'prd_radar_' + date_fname_template + fname_extension_tabular
radar_grid_fname_template = 'prd_radar_' + date_fname_template + fname_extension_grid
output_fname_template = 'prd_merged_' + leadtime_template + '_' + date_fname_template + fname_extension_tabular

In [10]:
variables_single_level = [
    "cloud_amount_of_total_cloud",
    "rainfall_accumulation-PT03H",
    "snowfall_accumulation-PT03H",
    "rainfall_rate",
    "snowfall_rate",
    "height_of_orography",
    "pressure_at_mean_sea_level",
]

variables_height_levels = [
    "cloud_amount_on_height_levels",
    "pressure_on_height_levels",
    "temperature_on_height_levels",
    "relative_humidity_on_height_levels",
    "wind_direction_on_height_levels",
    "wind_speed_on_height_levels",
    
]

In [11]:
num_periods = 10
start_ref_time = datetime.datetime(2020,2,14,12)
forecast_ref_time_range = [start_ref_time + datetime.timedelta(hours=6)*i1 for i1 in range(num_periods)]
leadtime_hours = 6
realizations_list = list(range(35))

In [12]:
dataset = 'mogreps-g'
subset = 'lev1'
forecast_ref_template = '{frt.year:04d}{frt.month:02d}{frt.day:02d}T{frt.hour:02d}00Z.nc.file'
fname_template = '{vt.year:04d}{vt.month:02d}{vt.day:02d}T{vt.hour:02d}00Z-PT{lead_time:04d}H00M-{var_name}.nc'

In [13]:
variables_to_extract = variables_height_levels + variables_single_level

In [14]:
path_lists_vars = {
    var_name: [f1 for f1 in mogreps_g_data_dir.iterdir() if var_name in str(f1)]
    for var_name in variables_to_extract
}


In [15]:
uk_bounds={'latitude':(50,58), 'longitude': (-6,2)}
xarray_select_uk = {k1: slice(*v1) for k1,v1 in uk_bounds.items()}

In [16]:
# these are modified compared to the actual improver thresholds, to remove the fuzziness of the range boundaries
improver_thresholds = {
"0.0": [0.0, 0.027],
"0.03": [0.027, 0.033],
"0.09": [0.033, 0.099],
"0.1": [0.099, 0.11],
"0.25": [0.11, 0.275],
"0.3": [0.275, 0.33],
"0.5": [0.33, 0.55],
"1.0": [0.55, 1.1],
"2.0": [1.1, 2.2],
"3.0": [2.2, 3.3],
"4.0": [3.3, 4.4],
"8.0": [4.4, 8.8],
"12.0": [8.8, 13.2],
"16.0": [13.2, 17.6],
"20.0": [17.6, 22.0],
"25.0": [22.0, 27.5],
"30.0": [27.5, 33.0],
"40.0": [33.0, 44.0],
"50.0": [44.0, 55.0],
"75.0": [55.0, 82.5],
"100.0": [82.5, 110.0],
"150.0": [110.0, 165.0],
"200.0": [165.0, 220.0]
}


## Load radar data

In [17]:
radar_days = [datetime.datetime(2020,2,14) + datetime.timedelta(days=d1) for d1 in range(4)]
radar_days

[datetime.datetime(2020, 2, 14, 0, 0),
 datetime.datetime(2020, 2, 15, 0, 0),
 datetime.datetime(2020, 2, 16, 0, 0),
 datetime.datetime(2020, 2, 17, 0, 0)]

In [18]:
radar_fname_template = 'composite_rainfall_{dt.year:04d}{dt.month:02d}{dt.day:02d}.nc'
radar_cube = iris.cube.CubeList([iris.load_cube(str(radar_data_dir / radar_fname_template.format(dt=dt1))) for dt1 in radar_days] ).concatenate_cube()

In [19]:
iris.coord_categorisation.add_hour(radar_cube, coord='time')
iris.coord_categorisation.add_day_of_year(radar_cube, coord='time')

Load a sample variable from MOGREPS-G to use for regridding radar data.

In [20]:
mogreps_g_example = iris.load_cube(
    str(mogreps_g_data_dir / fname_template.format(
        vt=forecast_ref_time_range[0] + datetime.timedelta(hours=leadtime_hours), 
        lead_time=leadtime_hours, 
        var_name=variables_single_level[0])),
    iris.Constraint(latitude=lambda cell1: uk_bounds['latitude'][0] < cell1 < uk_bounds['latitude'][1], 
                                                     longitude=lambda cell1: uk_bounds['longitude'][0] < cell1 < uk_bounds['longitude'][1], realization=0)
)


Aggregate the instantaneous rates to get an accumulation (this makes the assumption that the rates represent the accumulation for the 5 minute period).

In [21]:
coord_3hr = iris.coords.AuxCoord(radar_cube.coord('hour').points // 3,
                                long_name='3hr',
                                 units='hour',
                                )
radar_cube.add_aux_coord(coord_3hr, data_dims=0)
radar_agg_3hr = radar_cube.aggregated_by(['3hr', 'day_of_year'],iris.analysis.SUM)
radar_agg_3hr.add_aux_coord(iris.coords.AuxCoord([c1.bound[0] + datetime.timedelta(hours=3) for c1 in radar_agg_3hr.coord('time').cells()], long_name='model_accum_time', units='mm/h'), data_dims=0)

When we aggregate the data into 3 hour accumulations, for each hour each sum 12 observations, each of of which are giving us a rainfall rate in mm per hour. Our assumption is that represents the rainfall rate for a 5 minute period (60 minutes divided by 12 observations), so we want to translate this inot how many mm will fall in that 5 minute period. So we need to scale the value by 1/12 to get a 5 minute accumulation, which when aggregate all the 5 minute periods will give us correct 3 hour accumulations.

In [22]:
%%time
radar_agg_3hr.data = radar_agg_3hr.data * (1.0 /12.0)

CPU times: user 42.1 s, sys: 33 s, total: 1min 15s
Wall time: 1min 33s


In [23]:
radar_crs = radar_cube.coord_system().as_cartopy_crs()

/home/h01/shaddad/.conda/envs/prd_data_prep/lib/python3.8/site-packages/iris/coord_systems.py:525: UserWarning: The default value for the *approx* keyword argument to TransverseMercator will change from True to False after 0.18.
  return ccrs.TransverseMercator(


In [24]:
proj_y_grid = numpy.tile(radar_cube.coord('projection_y_coordinate').points.reshape(radar_cube.shape[1],1), [1, radar_cube.shape[2]])
proj_x_grid = numpy.tile(radar_cube.coord('projection_x_coordinate').points.reshape(1,radar_cube.shape[2]), [ radar_cube.shape[1],1])

In [25]:
ret_val = mogreps_g_example.coord_system().as_cartopy_crs().transform_points(
    radar_crs,
    proj_y_grid,
    proj_x_grid,
    )
lat_vals = ret_val[:,:,1]
lon_vals = ret_val[:,:,0]

In [26]:
lon_coord = iris.coords.AuxCoord(
    lon_vals,
    standard_name='longitude',
    units='degrees',
)
lat_coord = iris.coords.AuxCoord(
    lat_vals,
    standard_name='latitude',
    units='degrees',
)

In [27]:
radar_cube.add_aux_coord(lon_coord,[1,2])
radar_cube.add_aux_coord(lat_coord,[1,2])
radar_agg_3hr.add_aux_coord(lon_coord,[1,2])
radar_agg_3hr.add_aux_coord(lat_coord,[1,2])

In [28]:
# remove these coordinates as they interfere with subsequent calculations
radar_agg_3hr.remove_coord('model_accum_time')
radar_agg_3hr.remove_coord('forecast_reference_time')
radar_agg_3hr.remove_coord('hour')
radar_agg_3hr.remove_coord('day_of_year')
radar_agg_3hr.remove_coord('3hr')

In [29]:
radar_agg_3hr

Rainfall Rate Composite (mm/h),time,projection_y_coordinate,projection_x_coordinate
Shape,32,2175,1725
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
latitude,-,x,x
longitude,-,x,x
Scalar coordinates,,,forecast_period 0 second
Cell methods,,,


In [30]:
lat_mog_g_index  = numpy.zeros((radar_cube.shape[1],radar_cube.shape[2]))
lon_mog_g_index  = numpy.zeros((radar_cube.shape[1],radar_cube.shape[2]))

In [31]:
%%time
for i_lon, bnd_lon in enumerate(mogreps_g_example.coord('longitude').bounds):
    
    for i_lat, bnd_lat in enumerate(mogreps_g_example.coord('latitude').bounds):
        arr1, arr2 = numpy.where((lat_vals >= bnd_lat[0]) &
         (lat_vals < bnd_lat[1])&
         (lon_vals>= bnd_lon[0]) &
         (lon_vals < bnd_lon[1])
        )
        lon_mog_g_index[arr1, arr2] = i_lon
        lat_mog_g_index[arr1, arr2] = i_lat
     

CPU times: user 59.4 s, sys: 0 ns, total: 59.4 s
Wall time: 59.4 s


In [32]:
def compare_time(t1, t2):
    is_match = (t1.year == t2.year) and  (t1.month == t2.month) and  (t1.day == t2.day) and  (t1.hour== t2.hour) and  (t1.minute == t2.minute) 
    return is_match

In [33]:
bands_data = numpy.zeros([len(forecast_ref_time_range), mogreps_g_example.shape[0], mogreps_g_example.shape[1], len(improver_thresholds)])

In [41]:
max_rain_data = numpy.zeros([len(forecast_ref_time_range), mogreps_g_example.shape[0], mogreps_g_example.shape[1]])
mean_rain_data = numpy.zeros([len(forecast_ref_time_range), mogreps_g_example.shape[0], mogreps_g_example.shape[1]])

In [62]:
for i_time, fcst_ref_time in enumerate(forecast_ref_time_range):
    validity_time = fcst_ref_time + datetime.timedelta(hours=leadtime_hours)
    print(validity_time)
    radar_select_time = radar_agg_3hr.extract(iris.Constraint(time=lambda c1: compare_time(c1.bound[0], validity_time )))
    radar_data1 = radar_select_time.data.data
    masked_radar = numpy.ma.MaskedArray(
            radar_data1,
            radar_agg_3hr[0,:,:].data.mask)
    
    for i_lat in range(mogreps_g_example.shape[0]):
        for i_lon in range(mogreps_g_example.shape[1]):
            selected_cells = (~(radar_agg_3hr[0,:,:].data.mask)) & (lat_mog_g_index == i_lat)  & (lon_mog_g_index ==i_lon)
            masked_radar.mask = ~selected_cells
            radar_cells_in_mg = numpy.count_nonzero(selected_cells)
            if radar_cells_in_mg > 0:
                for imp_ix, (imp_key, imp_bounds) in enumerate(improver_thresholds.items()):
                    num_in_band = numpy.count_nonzero((radar_data1 >=  imp_bounds[0]) & (radar_data1 <= imp_bounds[1]) & selected_cells)
                    bands_data[i_time, i_lat, i_lon, imp_ix] = num_in_band / radar_cells_in_mg
                
                # calculate the max and average of all radar cells within each mogreps-g cell
                max_rain_data[i_time, i_lat, i_lon] = masked_radar.max()
                mean_rain_data[i_time, i_lat, i_lon] = (masked_radar.sum()) / radar_cells_in_mg    


2020-02-14 18:00:00
2020-02-15 00:00:00
2020-02-15 06:00:00
2020-02-15 12:00:00
2020-02-15 18:00:00
2020-02-16 00:00:00
2020-02-16 06:00:00
2020-02-16 12:00:00
2020-02-16 18:00:00
2020-02-17 00:00:00


In [63]:
mg_lat_coord = mogreps_g_example.coord('latitude')
mg_lon_coord = mogreps_g_example.coord('longitude')

In [64]:
band_coord = iris.coords.DimCoord(
    [float(b1) for b1 in improver_thresholds.keys()],
    bounds=list(improver_thresholds.values()),
    var_name='band',
    units='mm',
)

In [65]:
radar_cube.coord('time')

DimCoord(array([1581638400, 1581638700, 1581639000, ..., 1581983100, 1581983400,
       1581983700]), standard_name='time', units=Unit('seconds since 1970-01-01 00:00:00', calendar='gregorian'), var_name='time')

In [66]:
radar_time_coord = iris.coords.DimCoord(
    [(fcst_ref_time + datetime.timedelta(hours=leadtime_hours)).timestamp() for fcst_ref_time in forecast_ref_time_range],
    var_name='time',
    units=radar_cube.coord('time').units,
)

   

In [69]:
fraction_rain_band = iris.cube.Cube(
    data=bands_data, 
    dim_coords_and_dims=((radar_time_coord, 0),(mg_lat_coord, 1),(mg_lon_coord, 2),  (band_coord, 3)),
    units=None,
    var_name='fraction_in_band',
    long_name='Fraction radar rainfall cells in specified rain band',
)

In [70]:
max_rain_cube = iris.cube.Cube(
    data=max_rain_data, 
    dim_coords_and_dims=((radar_time_coord, 0),(mg_lat_coord, 1),(mg_lon_coord, 2),),
    units='mm',
    var_name='max_rain',
    long_name='maximum rain in radar cells within mogreps-g cell',
)

In [71]:
mean_rain_cube = iris.cube.Cube(
    data=mean_rain_data, 
    dim_coords_and_dims=((radar_time_coord, 0),(mg_lat_coord, 1),(mg_lon_coord, 2),),
    units='mm',
    var_name='mean_rain',
    long_name='average rain in radar cells within mogreps-g cell',
)

In [ ]:
cubelist_to_save = iris.cube.CubeList([fraction_rain_band, max_rain_cube, mean_rain_cube ])

In [ ]:
def cftime_to_datetime(input_cft):
    return datetime.datetime(input_cft.year,
                             input_cft.month,
                             input_cft.day,
                             input_cft.hour,
                             input_cft.minute,
                             input_cft.second,
                            )

In [ ]:
iris.save(radar_mggrid, 
          str(output_dir / radar_grid_fname_template.format(start=min([cftime_to_datetime(cell1.point) for cell1 in max_rain_cube.coord('time').cells()]),
                                                            end=max([cftime_to_datetime(cell1.point) for cell1 in max_rain_cube.coord('time').cells()])
                                                           )))

In [72]:
frac_df = xarray.DataArray.from_iris(fraction_rain_band).to_dataframe().reset_index()

In [73]:
imp_bands = list(improver_thresholds.keys())

In [74]:
radar_df = frac_df[frac_df['band'] ==  float(imp_bands[0])][['time','latitude','longitude','fraction_in_band']]
radar_df = radar_df.rename({'fraction_in_band': f'fraction_in_band_{imp_bands[0]}'},axis='columns')
radar_df

,time,latitude,longitude,fraction_in_band_0.0
0,2020-02-14 18:00:00,50.15625,-5.765625,0.622250
23,2020-02-14 18:00:00,50.15625,-5.484375,0.000000
46,2020-02-14 18:00:00,50.15625,-5.203125,0.023810
69,2020-02-14 18:00:00,50.15625,-4.921875,0.042755
92,2020-02-14 18:00:00,50.15625,-4.640625,0.251196
...,...,...,...,...
270365,2020-02-17 00:00:00,57.84375,0.703125,0.000000
270388,2020-02-17 00:00:00,57.84375,0.984375,0.000000
270411,2020-02-17 00:00:00,57.84375,1.265625,0.000000
270434,2020-02-17 00:00:00,57.84375,1.546875,0.000000


In [75]:
for band1 in imp_bands[1:]:
    df1 = frac_df[frac_df['band'] ==  float(band1)][['time','latitude','longitude','fraction_in_band']]
    df1 = df1.rename({'fraction_in_band': f'fraction_in_band_{band1}'},axis='columns')
    radar_df = pandas.merge(radar_df, df1, on=['time','latitude', 'longitude'])

In [76]:
radar_df = pandas.merge(radar_df, xarray.DataArray.from_iris(mean_rain_cube).to_dataframe().reset_index(), on=['time','latitude', 'longitude'])
radar_df = pandas.merge(radar_df, xarray.DataArray.from_iris(max_rain_cube).to_dataframe().reset_index(), on=['time','latitude', 'longitude'])

In [77]:
radar_df.to_csv(output_dir / radar_tab_fname_template.format(start=radar_df['time'].min(),
                                                                        end=radar_df['time'].min(),
                                                                       ))

## Create a dataset from MOGREPS-G data
Information on Met Office Ensmble forecasts - https://www.metoffice.gov.uk/research/weather/ensemble-forecasting#
Paper - https://www.metoffice.gov.uk/research/weather/ensemble-forecasting 

### Get the mapping of variable names 
Load some files and get the actual variable names.

In [42]:
fcst_ref_time = forecast_ref_time_range[0]
real1 = realizations_list[10]
validity_time = fcst_ref_time + datetime.timedelta(hours=leadtime_hours)

In [43]:
%%time
# load a cube for each variable in iris to get the actual variable name, and populate dictionary mapping from the var name in the file name to the variable as loaded into iris/xarray
file_to_var_mapping = {
    var_file_name: iris.load_cube(str(mogreps_g_data_dir / fname_template.format(vt=validity_time,
                                                                                 lead_time=leadtime_hours,
                                                                                 var_name=var_file_name))).name()
    for var_file_name in variables_single_level + variables_height_levels}
file_to_var_mapping

CPU times: user 776 ms, sys: 60.6 ms, total: 836 ms
Wall time: 2.22 s


{'cloud_amount_of_total_cloud': 'cloud_area_fraction',
 'rainfall_accumulation-PT03H': 'thickness_of_rainfall_amount',
 'snowfall_accumulation-PT03H': 'lwe_thickness_of_snowfall_amount',
 'rainfall_rate': 'rainfall_rate',
 'snowfall_rate': 'lwe_snowfall_rate',
 'height_of_orography': 'surface_altitude',
 'pressure_at_mean_sea_level': 'air_pressure_at_sea_level',
 'cloud_amount_on_height_levels': 'cloud_volume_fraction_in_atmosphere_layer',
 'pressure_on_height_levels': 'air_pressure',
 'temperature_on_height_levels': 'air_temperature',
 'relative_humidity_on_height_levels': 'relative_humidity',
 'wind_direction_on_height_levels': 'wind_from_direction',
 'wind_speed_on_height_levels': 'wind_speed'}

In [44]:
heights = iris.load_cube(str(mogreps_g_data_dir / fname_template.format(vt=validity_time,
                                                                                 lead_time=leadtime_hours,
                                                                                 var_name=variables_height_levels[0]))).coord('height').points

In [45]:
merge_coords = ['latitude', 'longitude', 'time', 'realization']

In [46]:
single_level_var_mappings = {v1: file_to_var_mapping[v1] for v1 in variables_single_level}
height_level_var_mappings = {v1: file_to_var_mapping[v1] for v1 in variables_height_levels}

In [78]:
def load_ds(ds_path, selected_bounds):
    try:
        subset1 = dict(selected_bounds)
        subset1['bnds'] = 0
        single_level_ds = xarray.load_dataset(ds_path).sel(**subset1)
    except KeyError as e1:
        single_level_ds = None
    return single_level_ds

In [79]:
%%time
ts_data_list = []
# gridded_data_list = []
for fcst_ref_time in forecast_ref_time_range:
    print(fcst_ref_time)
    validity_time = fcst_ref_time + datetime.timedelta(hours=leadtime_hours)
    single_level_ds = xarray.merge([load_ds(ds_path= mogreps_g_data_dir / fname_template.format(vt=validity_time,
                                                                                                lead_time=leadtime_hours,
                                                                                                var_name=var1),
                                            selected_bounds=xarray_select_uk,
                                           )
                                    for var1 in variables_single_level]
                                  )
    single_level_df = single_level_ds.to_dataframe().reset_index()

    height_levels_ds = xarray.merge([load_ds(ds_path=mogreps_g_data_dir / fname_template.format(vt=validity_time,
                                                                                                lead_time=leadtime_hours,
                                                                                                var_name=var1),
                                             selected_bounds=xarray_select_uk,
                                            )
                                     for var1 in variables_height_levels])
    hl_df_multirow = height_levels_ds.to_dataframe().reset_index()
    
    var_df_merged = []
    # heights_vars_marged = height_levels_df[height_levels_df.height==heights[0]][ merge_coords]
    for var1 in height_level_var_mappings.values():
        print(var1)
        # for h1 in heights:
        #     heights_vars_marged[f'{var1}_{h1:.1f}'] = list(height_levels_df[height_levels_df.height==h1][var1])
        var_at_heights = [hl_df_multirow[hl_df_multirow.height==h1][merge_coords + [var1]].rename({var1: f'{var1}_{h1:.1f}'}, axis='columns') for h1 in heights]
        var_df_merged += [functools.reduce(lambda x,y: x.merge(y, on=merge_coords), var_at_heights)]
    height_levels_df = functools.reduce(lambda x,y: x.merge(y, on=merge_coords), var_df_merged)    
    
    mogreps_g_single_ts_uk_df = single_level_df.merge(height_levels_df, on=merge_coords)
    mogreps_g_single_ts_uk_df
    
    mogreps_g_single_ts_uk_df = single_level_df.merge(height_levels_df, on=merge_coords)
    ts_data_list += [mogreps_g_single_ts_uk_df]
    ts_mogg_ds1 = xarray.merge([height_levels_ds, single_level_ds])
    ts_mogg_ds1.to_netcdf(output_dir / (
        'prd_mg_ts_'+ f'{validity_time.year:04d}{validity_time.month:02d}{validity_time.day:02d}{validity_time.hour:02d}{validity_time.minute:02d}' 
        + fname_extension_grid)
    )
    # gridded_data_list += [xarray.merge([height_levels_ds, single_level_ds])]

2020-02-14 12:00:00
cloud_volume_fraction_in_atmosphere_layer
air_pressure
air_temperature
relative_humidity
wind_from_direction
wind_speed
2020-02-14 18:00:00
cloud_volume_fraction_in_atmosphere_layer
air_pressure
air_temperature
relative_humidity
wind_from_direction
wind_speed
2020-02-15 00:00:00
cloud_volume_fraction_in_atmosphere_layer
air_pressure
air_temperature
relative_humidity
wind_from_direction
wind_speed
2020-02-15 06:00:00
cloud_volume_fraction_in_atmosphere_layer
air_pressure
air_temperature
relative_humidity
wind_from_direction
wind_speed
2020-02-15 12:00:00
cloud_volume_fraction_in_atmosphere_layer
air_pressure
air_temperature
relative_humidity
wind_from_direction
wind_speed
2020-02-15 18:00:00
cloud_volume_fraction_in_atmosphere_layer
air_pressure
air_temperature
relative_humidity
wind_from_direction
wind_speed
2020-02-16 00:00:00
cloud_volume_fraction_in_atmosphere_layer
air_pressure
air_temperature
relative_humidity
wind_from_direction
wind_speed
2020-02-16 06:00:00


In [ ]:
# prd_mogreps_grid_ds.to_netcdf(output_dir / mogreps_g_grid_fname_template.format(lt=leadtime_hours,
#                                                                                 start=prd_column_dataset['time'].min(),
#                                                                                 end=prd_column_dataset['time'].max(),
#                                                                                ))

In [80]:
prd_column_dataset = pandas.concat(ts_data_list)


In [81]:
prd_column_dataset

,realization,latitude,longitude,cloud_area_fraction,latitude_longitude,latitude_bnds,longitude_bnds,forecast_period,forecast_reference_time,time,...,wind_speed_2750.0,wind_speed_3000.0,wind_speed_3250.0,wind_speed_3500.0,wind_speed_3750.0,wind_speed_4000.0,wind_speed_4500.0,wind_speed_5000.0,wind_speed_5500.0,wind_speed_6000.0
0,0,50.15625,-5.765625,1.000000,-2147483647,50.0625,-5.90625,0 days 06:00:00,2020-02-14 12:00:00,2020-02-14 18:00:00,...,27.1250,27.3750,27.6250,27.7500,28.0000,28.3125,29.9375,31.7500,33.5625,34.8750
1,0,50.15625,-5.484375,1.000000,-2147483647,50.0625,-5.62500,0 days 06:00:00,2020-02-14 12:00:00,2020-02-14 18:00:00,...,25.7500,25.7500,25.8125,26.0000,26.4375,27.1250,29.0625,31.0625,32.8750,34.1250
2,0,50.15625,-5.203125,1.000000,-2147483647,50.0625,-5.34375,0 days 06:00:00,2020-02-14 12:00:00,2020-02-14 18:00:00,...,24.3125,24.3125,24.5000,24.9375,25.6250,26.4375,28.5625,30.7500,32.4375,33.4375
3,0,50.15625,-4.921875,1.000000,-2147483647,50.0625,-5.06250,0 days 06:00:00,2020-02-14 12:00:00,2020-02-14 18:00:00,...,23.5000,23.6250,24.0000,24.5000,25.3125,26.2500,28.5625,30.8125,32.3125,33.0000
4,0,50.15625,-4.640625,1.000000,-2147483647,50.0625,-4.78125,0 days 06:00:00,2020-02-14 12:00:00,2020-02-14 18:00:00,...,23.3750,23.5625,23.9375,24.6250,25.5000,26.6250,29.0625,31.6250,32.8125,32.9375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21163,34,57.84375,0.703125,0.796875,-2147483647,57.7500,0.56250,0 days 06:00:00,2020-02-16 18:00:00,2020-02-17 00:00:00,...,25.8125,24.9375,24.2500,23.7500,23.3750,23.1250,22.8750,22.6875,22.5000,22.8125
21164,34,57.84375,0.984375,1.000000,-2147483647,57.7500,0.84375,0 days 06:00:00,2020-02-16 18:00:00,2020-02-17 00:00:00,...,25.5625,24.8750,24.3750,24.0625,23.8750,23.6875,23.4375,23.0000,22.3750,22.3125
21165,34,57.84375,1.265625,0.921875,-2147483647,57.7500,1.12500,0 days 06:00:00,2020-02-16 18:00:00,2020-02-17 00:00:00,...,25.6250,25.0000,24.6250,24.4375,24.3125,24.1875,23.8125,23.0625,22.1875,21.7500
21166,34,57.84375,1.546875,0.937500,-2147483647,57.7500,1.40625,0 days 06:00:00,2020-02-16 18:00:00,2020-02-17 00:00:00,...,25.8125,25.1875,24.8125,24.5000,24.2500,24.0625,23.7500,22.9375,22.0000,21.3125


In [82]:
prd_column_dataset.to_csv(output_dir / mogreps_g_tab_fname_template.format(lt=leadtime_hours,
                                                                           start=prd_column_dataset['time'].min(),
                                                                           end=prd_column_dataset['time'].max(),
                                                                          ))

## Merging radar and model data
We have now created a table with radar data and a table with MOGREPS-G model data. We  now wantto merge them into a single. The steps are as follows
* trim the time periods so the data covers matching times
* do a merge on latitude, longitude and time. 
  * We don't merge on realization as done for merging the different model fields, as radar is not ensemble data. Instead, the type of merge chosen will insert the radar composite rainfall field for each row with with the correct time and place, so will appear multiple times for each realization present at that timestamp
* Output resulting table to disk.

In [83]:
prd_column_dataset.time.min(), prd_column_dataset.time.max()

(Timestamp('2020-02-14 18:00:00'), Timestamp('2020-02-17 00:00:00'))

In [84]:
prd_column_dataset.time

0       2020-02-14 18:00:00
1       2020-02-14 18:00:00
2       2020-02-14 18:00:00
3       2020-02-14 18:00:00
4       2020-02-14 18:00:00
                ...        
21163   2020-02-17 00:00:00
21164   2020-02-17 00:00:00
21165   2020-02-17 00:00:00
21166   2020-02-17 00:00:00
21167   2020-02-17 00:00:00
Name: time, Length: 211680, dtype: datetime64[ns]

In [85]:
# radar_acc_regrid_df = radar_acc_regrid_df[(radar_acc_regrid_df['time'] >= prd_column_dataset.time.min()) & (radar_acc_regrid_df['time'] <= prd_column_dataset.time.max())]
# radar_acc_regrid_df

In [86]:
prd_merged_mogreps_radar = prd_column_dataset.merge(radar_df, on=['latitude', 'longitude','time'], how='inner')
prd_merged_mogreps_radar

,realization,latitude,longitude,cloud_area_fraction,latitude_longitude,latitude_bnds,longitude_bnds,forecast_period,forecast_reference_time,time,...,fraction_in_band_25.0,fraction_in_band_30.0,fraction_in_band_40.0,fraction_in_band_50.0,fraction_in_band_75.0,fraction_in_band_100.0,fraction_in_band_150.0,fraction_in_band_200.0,mean_rain,max_rain
0,0,50.15625,-5.765625,1.000000,-2147483647,50.0625,-5.90625,0 days 06:00:00,2020-02-14 12:00:00,2020-02-14 18:00:00,...,0.0,0.000001,0.000001,0.0,0.000003,0.0,0.0,0.0,0.370506,63.59375
1,1,50.15625,-5.765625,1.000000,-2147483647,50.0625,-5.90625,0 days 06:00:00,2020-02-14 12:00:00,2020-02-14 18:00:00,...,0.0,0.000001,0.000001,0.0,0.000003,0.0,0.0,0.0,0.370506,63.59375
2,2,50.15625,-5.765625,1.000000,-2147483647,50.0625,-5.90625,0 days 06:00:00,2020-02-14 12:00:00,2020-02-14 18:00:00,...,0.0,0.000001,0.000001,0.0,0.000003,0.0,0.0,0.0,0.370506,63.59375
3,3,50.15625,-5.765625,1.000000,-2147483647,50.0625,-5.90625,0 days 06:00:00,2020-02-14 12:00:00,2020-02-14 18:00:00,...,0.0,0.000001,0.000001,0.0,0.000003,0.0,0.0,0.0,0.370506,63.59375
4,4,50.15625,-5.765625,1.000000,-2147483647,50.0625,-5.90625,0 days 06:00:00,2020-02-14 12:00:00,2020-02-14 18:00:00,...,0.0,0.000001,0.000001,0.0,0.000003,0.0,0.0,0.0,0.370506,63.59375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211675,30,57.84375,1.828125,0.937500,-2147483647,57.7500,1.68750,0 days 06:00:00,2020-02-16 18:00:00,2020-02-17 00:00:00,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000
211676,31,57.84375,1.828125,0.984375,-2147483647,57.7500,1.68750,0 days 06:00:00,2020-02-16 18:00:00,2020-02-17 00:00:00,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000
211677,32,57.84375,1.828125,1.000000,-2147483647,57.7500,1.68750,0 days 06:00:00,2020-02-16 18:00:00,2020-02-17 00:00:00,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000
211678,33,57.84375,1.828125,0.859375,-2147483647,57.7500,1.68750,0 days 06:00:00,2020-02-16 18:00:00,2020-02-17 00:00:00,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000


Looking at the results of the merge, we see that we have different values for model output for different realisations at the same time and location, but all of those datapoints will have the same value for radar rainfall accumulation.

In [ ]:
prd_merged_mogreps_radar[(prd_merged_mogreps_radar['time'] == '2020-02-16T015:00') &
                         (prd_merged_mogreps_radar['latitude'] == 50.15625) &
                         (prd_merged_mogreps_radar['longitude'] == -5.765625) 
                        ][['latitude','longitude','realization', 'time','air_temperature_5.0','rainfall_rate_composite']]

### Output to Tabular data format

In [87]:
start_dt = prd_merged_mogreps_radar['time'].min()
end_dt = prd_merged_mogreps_radar['time'].max()

In [88]:
output_fname = output_fname_template.format(lt=leadtime_hours,
                                            start=start_dt,
                                            end=end_dt,
                                           )
output_path = output_dir / output_fname
print(output_path)
prd_merged_mogreps_radar.to_csv(output_path)

/scratch/shaddad/precip_rediagnosis/storm_dennis_6hr_lt/prd_merged_006H_20200214T1800Z_20200217T0000Z.csv


In [91]:
prd_merged_mogreps_radar['time'].value_counts()

2020-02-14 18:00:00    21168
2020-02-15 00:00:00    21168
2020-02-15 06:00:00    21168
2020-02-15 12:00:00    21168
2020-02-15 18:00:00    21168
2020-02-16 00:00:00    21168
2020-02-16 06:00:00    21168
2020-02-16 12:00:00    21168
2020-02-16 18:00:00    21168
2020-02-17 00:00:00    21168
Name: time, dtype: int64

In [ ]:
#TODO:add parquet to conda environment and save as parquet format
#prd_merged_mogreps_radar.to_parquet(output_path)